In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from dataset_build.clf_dtSet_hugo import build_purity_classification_dataset

In [4]:
from sklearn.model_selection import ParameterGrid
import time

# Paramètres à optimiser
n_shots_values = [10, 50, 100, 500, 1000, 5000]
svc_params = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],  # Pas de 'poly' par défaut
    'gamma': ['scale', 'auto'],
    'degree': [2]  # Paramètre de degré pour polynomial
}

# Créer la grille des paramètres
svc_param_grid = list(ParameterGrid(svc_params))

# Ajouter manuellement le kernel polynomial de degré 2
svc_param_grid_extended = []
for params in svc_param_grid:
    svc_param_grid_extended.append(params)

# Ajouter les combinaisons avec polynomial de degré 2
for C in [0.1, 1, 10]:
    for gamma in ['scale', 'auto']:
        svc_param_grid_extended.append({
            'C': C,
            'kernel': 'poly',
            'gamma': gamma,
            'degree': 2
        })

svc_param_grid = svc_param_grid_extended

# Résultats
results_grid = []
total_combinations = len(n_shots_values) * len(svc_param_grid)

print("="*100)
print("GRIDSEARCH: OPTIMISATION CONJOINTE DE n_shots ET PARAMÈTRES SVC")
print("="*100)
print(f"\nParamètres SVC: {len(svc_param_grid)} combinaisons")
print(f"Valeurs n_shots: {len(n_shots_values)} valeurs")
print(f"Total d'itérations: {total_combinations}\n")
print(f"Configuration:")
print(f"  • n_states_total = 5000")
print(f"  • mixed_proportion = 0.02")
print(f"  • Kernels: linear, rbf, poly(degree=2)")
print(f"  • Métrique: F1-Score (weighted)")
print(f"  • Cross-validation: 5-fold")
print("="*100 + "\n")

iteration = 0
start_time = time.time()

for n_shots in n_shots_values:
    print(f"\n[n_shots = {n_shots:5d}] Génération dataset et GridSearch SVC...")
    print("-" * 100)
    
    # Générer le dataset avec ce n_shots
    df_temp, _, y_temp = build_purity_classification_dataset(
        n_shots=n_shots,
        n_states_total=5000,
        mixed_proportion=0.02
    )
    
    X_temp_data = df_temp[['X_mean', 'Y_mean', 'Z_mean']]
    X_temp_scaled = StandardScaler().fit_transform(X_temp_data)
    
    # Tester chaque combinaison de paramètres SVC
    best_f1_for_n_shots = 0
    best_config_for_n_shots = None
    
    for params in svc_param_grid:
        iteration += 1
        
        # Créer et évaluer le modèle
        svc_model = SVC(**params)
        
        from sklearn.model_selection import cross_val_score
        cv_scores = cross_val_score(
            svc_model, X_temp_scaled, y_temp, 
            cv=5, scoring='f1_weighted'
        )
        
        # Enregistrer les résultats
        result = {
            'n_shots': n_shots,
            'C': params['C'],
            'kernel': params['kernel'],
            'gamma': params['gamma'],
            'degree': params.get('degree', None),
            'mean_f1': cv_scores.mean(),
            'std_f1': cv_scores.std()
        }
        results_grid.append(result)
        
        # Afficher chaque exécution
        degree_str = f", degree={params.get('degree', 'N/A')}" if params['kernel'] == 'poly' else ""
        print(f"  n_shots={n_shots:5d} | C={params['C']:4.1f} | kernel={params['kernel']:6s} | gamma={params['gamma']:6s}{degree_str} | F1={cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
        
        # Tracker le meilleur pour ce n_shots
        if cv_scores.mean() > best_f1_for_n_shots:
            best_f1_for_n_shots = cv_scores.mean()
            best_config_for_n_shots = result
    
    # Afficher le meilleur pour ce n_shots
    print(f"  → MEILLEUR POUR n_shots={n_shots}: F1={best_f1_for_n_shots:.4f} (C={best_config_for_n_shots['C']}, kernel={best_config_for_n_shots['kernel']}, gamma={best_config_for_n_shots['gamma']})")

# Créer un DataFrame avec tous les résultats
results_grid_df = pd.DataFrame(results_grid)

# Trouver le meilleur résultat global
best_idx = results_grid_df['mean_f1'].idxmax()
best_result = results_grid_df.loc[best_idx]

elapsed_time = time.time() - start_time

print("\n" + "="*100)
print("RÉSULTATS DU GRIDSEARCH - MEILLEURE CONFIGURATION GLOBALE")
print("="*100)
print(f"\n✓ MEILLEUR n_shots: {int(best_result['n_shots'])}")
print(f"\n✓ MEILLEURS PARAMÈTRES SVC:")
print(f"  • C: {best_result['C']}")
print(f"  • kernel: {best_result['kernel']}")
print(f"  • gamma: {best_result['gamma']}")
if best_result['degree'] is not None and best_result['kernel'] == 'poly':
    print(f"  • degree: {int(best_result['degree'])}")
print(f"\n✓ F1-Score (weighted): {best_result['mean_f1']:.4f} ± {best_result['std_f1']:.4f}")
print(f"\n✓ Temps d'exécution total: {elapsed_time:.2f} secondes")
print("="*100)


GRIDSEARCH: OPTIMISATION CONJOINTE DE n_shots ET PARAMÈTRES SVC

Paramètres SVC: 18 combinaisons
Valeurs n_shots: 6 valeurs
Total d'itérations: 108

Configuration:
  • n_states_total = 5000
  • mixed_proportion = 0.02
  • Kernels: linear, rbf, poly(degree=2)
  • Métrique: F1-Score (weighted)
  • Cross-validation: 5-fold


[n_shots =    10] Génération dataset et GridSearch SVC...
----------------------------------------------------------------------------------------------------
  n_shots=   10 | C= 0.1 | kernel=linear | gamma=scale  | F1=0.9701 ± 0.0000
  n_shots=   10 | C= 0.1 | kernel=rbf    | gamma=scale  | F1=0.9701 ± 0.0000
  n_shots=   10 | C= 0.1 | kernel=linear | gamma=auto   | F1=0.9701 ± 0.0000
  n_shots=   10 | C= 0.1 | kernel=rbf    | gamma=auto   | F1=0.9701 ± 0.0000
  n_shots=   10 | C= 1.0 | kernel=linear | gamma=scale  | F1=0.9701 ± 0.0000
  n_shots=   10 | C= 1.0 | kernel=rbf    | gamma=scale  | F1=0.9701 ± 0.0000
  n_shots=   10 | C= 1.0 | kernel=linear | gamma=auto  

In [ ]:

from dataset_build.saint_dtSet import generate_qubit_tomography_dataset_base, perform_mle_tomography

def test_pipeline():
    # ---------------------------------------------------------
    # 1. Génération du Dataset (avec Décohérence pour avoir des états mixtes)
    # ---------------------------------------------------------
    print(">>> 1. Génération du dataset de test...")
    n_shots_test = 200
    
    # On demande de la décohérence (include_decoherence=True)
    # decoherence_level=0.9 implique que certains vecteurs seront très courts (très mixtes)
    df = generate_qubit_tomography_dataset_base(
        n_states=5,                 # Petit nombre pour bien voir les logs
        n_shots=n_shots_test,
        mode="finite_shots",        # Avec bruit statistique
        include_decoherence=True,   # IMPORTANT : Crée des états mixtes
        decoherence_level=0.9,      
        random_state=42
    )
    
    # Calcul du rayon réel (pureté réelle) pour vérification
    df['r_real'] = np.sqrt(df['X_real']**2 + df['Y_real']**2 + df['Z_real']**2)
    
    print("   Dataset généré. Voici les rayons réels (Pureté) des 5 états :")
    print("   ", df['r_real'].values.round(3))
    print("-" * 50)

    # ---------------------------------------------------------
    # 2. Exécution du MLE (Matrice Densité)
    # ---------------------------------------------------------
    print(">>> 2. Lancement du MLE...")
    
    # Note : Pas besoin de passer n_shots, il est détecté automatiquement dans le df
    df_result, duration = perform_mle_tomography(df)
    
    print(f"   Terminé en {duration:.4f} secondes.")
    print("-" * 50)

    # ---------------------------------------------------------
    # 3. Analyse des Résultats
    # ---------------------------------------------------------
    print(">>> 3. Comparaison Réel vs MLE :\n")
    
    # On affiche une comparaison côte à côte pour chaque état
    for i, row in df_result.iterrows():
        print(f"État #{i} :")
        
        # Comparaison des Puretés (Rayons)
        # Si le MLE marche bien pour les états mixtes, r_mle doit être proche de r_real
        # et non bloqué à 1.0
        r_real = row['r_real']
        r_mle  = row['r_mle']
        
        # Comparaison des Vecteurs
        print(f"  [Pureté r] Réel: {r_real:.3f}  vs  MLE: {r_mle:.3f}")
        print(f"  [Coord X ] Réel: {row['X_real']:.3f}  vs  MLE: {row['X_mle']:.3f}")
        print(f"  [Coord Z ] Réel: {row['Z_real']:.3f}  vs  MLE: {row['Z_mle']:.3f}")
        
        # Distance (Erreur)
        dist = np.sqrt(
            (row['X_real'] - row['X_mle'])**2 +
            (row['Y_real'] - row['Y_mle'])**2 +
            (row['Z_real'] - row['Z_mle'])**2
        )
        print(f"  => Distance Euclidienne : {dist:.4f}")
        print("")

if __name__ == "__main__":
    test_pipeline()

>>> 1. Génération du dataset de test...
   Dataset généré. Voici les rayons réels (Pureté) des 5 états :
    [0.354 0.742 0.523 0.199 0.302]
--------------------------------------------------
>>> 2. Lancement du MLE...
   Terminé en 0.3018 secondes.
--------------------------------------------------
>>> 3. Comparaison Réel vs MLE :

État #0 :
  [Pureté r] Réel: 0.354  vs  MLE: 0.297
  [Coord X ] Réel: -0.172  vs  MLE: -0.160
  [Coord Z ] Réel: 0.257  vs  MLE: 0.220
  => Distance Euclidienne : 0.0652

État #1 :
  [Pureté r] Réel: 0.742  vs  MLE: 0.725
  [Coord X ] Réel: -0.041  vs  MLE: 0.167
  [Coord Z ] Réel: -0.575  vs  MLE: -0.589
  => Distance Euclidienne : 0.2223

État #2 :
  [Pureté r] Réel: 0.523  vs  MLE: 0.497
  [Coord X ] Réel: -0.510  vs  MLE: -0.400
  [Coord Z ] Réel: -0.113  vs  MLE: -0.181
  => Distance Euclidienne : 0.2975

État #3 :
  [Pureté r] Réel: 0.199  vs  MLE: 0.210
  [Coord X ] Réel: -0.189  vs  MLE: -0.200
  [Coord Z ] Réel: -0.005  vs  MLE: 0.050
  => Distance